In [19]:
# Python natives
import os

os.chdir("/home/tim/Development/OCPPM/")
import lightgbm as lgb
import pprint
import numpy as np
import pandas as pd
import re
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from custom_utilities import evaluation_helpers
from sklearn.preprocessing import StandardScaler

In [2]:
oft_in_file = "data/CS/feature_encodings/baselines/OFT/objects.csv"

In [13]:
objects_df = pd.read_csv(oft_in_file, sep=";")
objects_df.head()

,oa1,oa2,oa3,oa4,oa5,oa6,oa7,oa8,oa9,oa10,oa11,oa12,oa13,oa14,oa15,oa16,ea4
0,0.368881,40567,134640,298154,-2.565395,400806,404283,420092,280069,228280,256802,265745,22897,0.0,50298,280149,0.0
1,-1.129970,40567,27852,298154,-2.565395,400806,404283,420092,280069,32709,163528,265745,22897,0.0,347082,280149,0.0
2,0.368881,236144,134640,298154,-0.481001,400806,404283,420092,280069,102653,163528,41856,364012,0.0,6647,280149,3.0
3,0.368881,236144,258390,298154,1.082294,400806,404283,420092,280069,6398,256802,265745,364012,0.0,347082,280149,0.0
4,0.368881,236144,134640,298154,1.082294,400806,404283,420092,280069,6398,256802,265745,364012,0.0,50298,280149,0.0


In [14]:
# oft =oft[oft["ea4"] != 0]
# oft["ea4"] = oft["ea4"] - 1

scaler = StandardScaler()
oft_values = scaler.fit_transform(objects_df.drop(columns=["ea4"]).values)
oft = pd.DataFrame(
    oft_values, columns=[col for col in objects_df.columns if col != "ea4"]
)
oft["ea4"] = objects_df["ea4"]
oft.head()

,oa1,oa2,oa3,oa4,oa5,oa6,oa7,oa8,oa9,oa10,oa11,oa12,oa13,oa14,oa15,oa16,ea4
0,0.581041,-1.578327,-0.936985,0.640128,-3.432927,0.223806,0.202627,0.043365,0.70907,0.869652,0.792789,0.744162,-2.505123,0.013777,-2.065711,0.700574,0.0
1,-0.796820,-1.578327,-2.389081,0.640128,-3.432927,0.223806,0.202627,0.043365,0.70907,-1.404554,-1.228912,0.744162,-2.505123,0.013777,0.460252,0.700574,0.0
2,0.581041,0.843833,-0.936985,0.640128,-0.814502,0.223806,0.202627,0.043365,0.70907,-0.591207,-1.228912,-1.778615,0.395221,0.013777,-2.437230,0.700574,3.0
3,0.581041,0.843833,0.745758,0.640128,1.149316,0.223806,0.202627,0.043365,0.70907,-1.710513,0.792789,0.744162,0.395221,0.013777,0.460252,0.700574,0.0
4,0.581041,0.843833,-0.936985,0.640128,1.149316,0.223806,0.202627,0.043365,0.70907,-1.710513,0.792789,0.744162,0.395221,0.013777,-2.065711,0.700574,0.0


In [15]:
# make train test split
X, y = (
    oft.drop(columns=["ea4"]),
    oft["ea4"],
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

train_data = lgb.Dataset(
    X_train,
    label=y_train,
)
valid_data = lgb.Dataset(
    X_valid,
    label=y_valid,
)

In [17]:
oft["ea4"].value_counts()

0.0    297420
6.0     42023
4.0     35565
5.0     30988
3.0      6133
1.0      5069
2.0      3684
Name: ea4, dtype: int64

In [18]:
params = {
    "num_threads": 4,
    "objective": "multiclass",
    "num_class": 7,
    "metric": ["multi_logloss"],
    "num_boost_round": 100,
    "stopping_rounds": 100,
}

bst = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    callbacks=[lgb.early_stopping(params["stopping_rounds"])],
)

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.361121 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 105
[LightGBM] [Info] Number of data points in the train set: 336705, number of used features: 16
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Info] Start training from score -0.347365
[LightGBM] [Info] Start training from score -4.420490
[LightGBM] [Info] Start training from score -4.734694
[LightGBM] [Info] Start training from score -4.217801
[LightGBM] [Info] Start training from score -2.472607
[LightGBM] [Info] Start training from score -2.608444
[LightGBM] [Info] Start training from score -2.303640
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.707429


In [20]:
y_train_probs = bst.predict(X_train)
y_valid_probs = bst.predict(X_valid)

y_train_preds = np.apply_along_axis(
    evaluation_helpers.get_preds_from_probs, axis=1, arr=y_train_probs
)
y_valid_preds = np.apply_along_axis(
    evaluation_helpers.get_preds_from_probs, axis=1, arr=y_valid_probs
)

In [21]:
eval_results = evaluation_helpers.get_classification_evaluation(
    y_train, y_train_preds, y_valid, y_valid_preds
) | {"params": params}

pprint.pprint(eval_results)

{'params': {'metric': ['multi_logloss'],
            'num_boost_round': 100,
            'num_class': 7,
            'num_threads': 4,
            'objective': 'multiclass',
            'stopping_rounds': 100},
 'train': {'confusion_matrix': array([[225875,    385,      3,      1,   5087,   1437,   5110],
       [  3126,    391,      0,      0,    286,     69,    178],
       [  2422,      7,     21,      0,    271,     69,    168],
       [  3789,      3,      0,     15,    702,    117,    334],
       [ 17903,      2,      0,      2,   7132,   1126,   2241],
       [ 14817,      3,      2,      0,   3108,   3322,   3546],
       [ 14009,      3,      1,      1,   3448,   1349,  14824]]),
           'report': {'0.0': {'f1-score': 0.8690190616710175,
                              'precision': 0.8011427922863295,
                              'recall': 0.9494615339347116,
                              'support': 237898},
                      '1.0': {'f1-score': 0.16143682906688686,
   

In [ ]:
# Validation scores:
# including class '0': 79% accuracy (with highest prior of 0.7 for class '0')
# excluding class '0': 58% accuracy (with highest prior of 0.34 for class '5', which is actually 6)